In [1]:
import pandas as pd
import numpy as np

#Importing all processed data
diag = pd.read_csv('Data/data_set/diagnosis.csv', low_memory = False)
demo = pd.read_csv('Data/data_set/demo_df.csv', low_memory = False)
med = pd.read_csv('Data/data_set/med_df.csv', low_memory = False)
prob = pd.read_csv('Data/data_set/prob_df.csv', low_memory = False)
proc = pd.read_csv('Data/data_set/proc_df.csv', low_memory = False)
soc = pd.read_csv('Data/data_set/soc_df.csv', low_memory = False)
vitals = pd.read_csv('Data/data_set/vitals_df.csv', low_memory = False)
diagnostics_qual = pd.read_csv('Data/data_set/diag_qual_df.csv', low_memory = False)
diagnostics_quant = pd.read_csv('Data/data_set/diag_quant_df.csv', low_memory = False)

#Resetting index to patient_num
diag.rename(columns={'Unnamed: 0':'patient_num'},inplace= True)
diag.set_index('patient_num',inplace= True)
demo.rename(columns={'Unnamed: 0':'patient_num'},inplace= True)
demo.set_index('patient_num',inplace= True)
med.rename(columns={'Unnamed: 0':'patient_num'},inplace= True)
med.set_index('patient_num',inplace= True)
prob.rename(columns={'Unnamed: 0':'patient_num'},inplace= True)
prob.set_index('patient_num',inplace= True)
proc.rename(columns={'Unnamed: 0':'patient_num'},inplace= True)
proc.set_index('patient_num',inplace= True)
soc.rename(columns={'Unnamed: 0':'patient_num'},inplace= True)
soc.set_index('patient_num',inplace= True)
vitals.rename(columns={'Unnamed: 0':'patient_num'},inplace= True)
vitals.set_index('patient_num',inplace= True)
diagnostics_qual.rename(columns={'Unnamed: 0':'patient_num'},inplace= True)
diagnostics_qual.set_index('patient_num',inplace= True)
diagnostics_quant.rename(columns={'Unnamed: 0':'patient_num'},inplace= True)
diagnostics_quant.set_index('patient_num',inplace= True)

#Merge all of the data
df = pd.merge(diag,demo, left_index= True,right_index= True, how = 'inner')
df = pd.merge(df,med, left_index= True,right_index= True, how = 'inner')
df = pd.merge(df,prob, left_index= True,right_index= True, how = 'inner')
df = pd.merge(df,proc, left_index= True,right_index= True, how = 'inner')
df = pd.merge(df,soc, left_index= True,right_index= True, how = 'inner')
df = pd.merge(df,vitals, left_index= True,right_index= True, how = 'inner')
df = pd.merge(df,diagnostics_qual, left_index= True,right_index= True, how = 'inner')
df = pd.merge(df,diagnostics_quant, left_index= True,right_index= True, how = 'inner')

#Delete imported data to save space
del diag
del demo
del diagnostics_qual
del diagnostics_quant
del med
del prob
del soc
del proc
del vitals

In [2]:
#NAN resolution
df.dropna(axis = 1, thresh=.5*df.shape[0],inplace = True)
df.dropna(axis = 0, thresh=.5*df.shape[1],inplace = True)
df.dropna(axis = 0, subset = ['LOS'], inplace = True)

In [19]:
df.shape

(11235, 36417)

In [3]:
df.shape

(11235, 35943)

In [33]:
#Add indicator to show which values are missing
col_w_na = (df.isna().sum()[df.isna().sum()>0]/df.shape[0]).index
for col in col_w_na:
    df[col + '_na'] = pd.isna(df.loc[:,col]).astype('int')

In [39]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=1, weights="distance")
knnimputed_df = pd.DataFrame(data = imputer.fit_transform(df),columns= df.columns,index = df.index)

In [46]:
knnimputed_df.to_csv('Data/final_dataset/df.csv')